In [ ]:
#pip install pymongo
from pymongo import MongoClient
# pprint library is used to make the output look more pretty
from pprint import pprint
from random import randint
client = MongoClient("localhost:27017")
#let's connect to greatlearning database, and insert 1000 documents containing product name, company name and rating

db = client['greatlearning']
productcollection = db['product_feedback']

#
productlist = []
product_name = ['Mobile','TV','Washing machine', 'Refrigerator', 'Microwave Oven','Induction cooker','AC']
company_name = ['LG', 'Samsung', 'Bosch','Siemens','Whirlpool','Electrolux','Haier','Videocon']
for x in range(1, 1001):
    sale = {
        'name' : product_name[randint(0, (len(product_name)-1))],
        'rating' : randint(1, 5),
        'brand' : company_name[randint(0, (len(company_name)-1))] 
    }
    productlist.append(sale)
    
productcollection.insert_many(productlist)



Once, we have inserted 1000 documents then we can peform different CRUD operations using PyMongo.
Let say , we want to find those products which have recived 5/5 rating.

In [ ]:
rating5reviews= productcollection.find({"rating":5})
for  review in rating5reviews:
    print(review)

Let's count how many of reviws feedback have given 5/5 of rating and which brand has maximum 5/5 of ratings

In [ ]:
pprint(productcollection.count_documents({"rating":5}))

#to findout which brand has got maximum 5/5 rating we need to use aggregation pipeline
from bson.son import SON
pipeline = [{"$match": {"rating": 5}},{"$group": {"_id": "$brand", "count": {"$sum": 1}}},{"$sort": SON([("count", -1), ("_id", -1)])}]
pprint(list(productcollection.aggregate(pipeline)))


Now, same aggregation logic,we will implement using MapRedcue framework. Here we will define map and reduce functions to also count the number of 5rating received by a particular brand, across the entire collection.

Our map function just emits a single (brand, 1) pair if rating is 5

In [ ]:
from bson.code import Code
mapper = Code("""
              function () {
                   if(this.rating == 5)
                     emit(this.brand, 1);
                 
               }
               """)

reducer = Code("""
                function (key, values) {
                  var total = 0;
                  for (var i = 0; i < values.length; i++) {
                    total += values[i];
                  }
                  return total;
                }
                """)


productcollection.map_reduce(mapper, reducer,"myresults")
result = db['myresults'].find()
for doc in result:
   pprint(doc)

let's update those records where product name is AC to "Air conditioner"

In [ ]:
productcollection.update_many({"name": "AC"},{"$set": {"name": "Air conditioner"} })
for doc in productcollection.find({"name": "Air conditioner"}):
   pprint(doc)